## Praxiskurs Termin 1

Anmerkung:
Eine detaillierte Einführung zu DataFrames, Plotting, Piping etc. könnt ihr auf der Website des [Julia](https://github.com/JohannesNaegele/Programmierkurs-Julia) Kurses finden oder auch in folgendem Kurs zur Datenbearbeitung in Julia: [juliadatascience.io](https://juliadatascience.io/)

#### Einführung in die Datenanalyse 
Als Beispieldatensatz betrachten wir die Entwckelung der globalen Rohstoffpreise. Die Daten sind auf der [Website](https://www.worldbank.org/en/research/commodity-markets) der Worldbank verfügbar (monatliche Preise). Zum Einlesen von Exeldateien und der Konvertierung in den Datentyp `DataFrame`, benötigen wir in Julia zunächst die folgenden beiden packages.

In [ ]:
using DataFrames
using XLSX

In [ ]:
file_path = "CMO-Historical-Data-Monthly.xlsx"
sheet_name = "Monthly Prices"

# Read the Excel file
xlsx_data = XLSX.readdata(file_path, sheet_name, "A5:BT754")

col_names = xlsx_data[1, :]
df = DataFrame(xlsx_data[4:end, :], Symbol.(col_names))

first(df, 10)

Zunächst geben wir der ersten Spalte in dem DataFrame den namen date um in Zukunft einfacher darauf zurückzugreifen und ändern alle fehlenden Werte in dem DataFrame die durch den String ```"…"``` oder ````".."```` gekennzeichnet sind in den Datentyp ````missing ````um  


In [ ]:
#1 Umbennung der ersten Spalte 
rename!(df, 1 => "date")

#2 Konvertierung in den Datentyp missing 
function transform_missing!(df)
    for col in names(df)
        if_missing_string(x) = x in ["..", "…"] ? missing : x
        transform!(df, col => ByRow(if_missing_string) => col)
    end
end

transform_missing!(df)

first(df,10)

Als letztes wollen wir die Spalte mit den Daten in ein von Julia unterstütztes Datumsformat umwandeln, um später auch die Zeitpunkte sinnvoll miteinander vergleichen zu können. Hierfür verwenden wir das package ````Date````.


In [ ]:
using Dates 

function transform_dates(date)
    date = String.(date)
    new_date = replace(date,"M" => "-")
    return Date(new_date,dateformat"yyyy-mm")
end 

transform!(df, :date  => ByRow(transform_dates) ,renamecols = false)

first(df,10)

Wir können auch gezielt Abfragen an unseren Dataframe stellen un verändern. Als Beispiel lassen wir uns alle Ölpreise für die Monate ausgeben in denen die Sorte Brent unter 100$ lagen und geben diese geordnet nach Datum aus (beginnend mit dem neusten Datum)


In [ ]:
df_oil_brent = sort(subset(df, "Crude oil, Brent" => x -> x .<= 100, skipmissing=true), :date, rev=true)

println(first(df_oil_brent, 10))

Falls in der Spalte Datenpunkte vom Typ ``missing`` auftauchen kann die Function ``coalesce `` hilfreich sein 



In [ ]:
df_oil_brent = sort(subset(transform(df, "Crude oil, WTI" => (x -> coalesce.(x, 0)), renamecols=false), "Crude oil, WTI" => x -> x .<= 100), :date, rev=true)

println(first(df_oil_brent, 10))

Der obige Code kann sehr schnell sehr unübersichtlich werden, sobald man etwas komplexere Abfragen an den Datensatz stellt. Deswegen empfielt es sich, das package ``Pipe`` zu benutzen. 

In [ ]:
using Pipe

df_oil_brent = @pipe df |>
    subset(_, "Crude oil, Brent" => (x -> x .<= 100), skipmissing=true) |>
    sort(_, :date, rev=true) |>
    println(first(_, 10))

### Aufgabe 1 

1. Gib die zehn höchsten Preise der Kaffeesorte Arabica aus mit dem jeweiligen Datum.

2. Erstellt im Datensatz eine neue Spalte, die das Verhältnis der Preise der Kaffeesorten Arabica und Robusta beinhaltet. Was ist hier das Maximum und das Minium? Hinweis: benutze ``transform``, ``maximum``, ``minimum``.

3. Bestimmt für Rohölsorten Brent und WTI den Kalendermonat (Januar bis Dezember) für den der Preis über die Jahre hinweg den höchsten Durchschnitt Datensatz hat.

Hinweise: 
- mit der Funktion ``month`` aus dem package Dates erhält man den Monat aus einem Datum: `2020-02-01` wird zu `"02"`
- benutze ``groupby`` und ``combine``
- aus dem Package ``Statistics`` erhaltet ihr eine vordefinierte ``mean`` Funktion


#### Einführung in Plots 
Wir können Daten mithilfe Standardpakets `` Plots`` graphisch darstellen. Hierfür geben wir uns zunächst die Preise der Rohölsorte Brent aus und formatieren diese angemessen.

In [ ]:
using Plots

x = df.date
y = df[:, "Crude oil, Brent"]

# Erstelle für einen Array der beginnend mit dem ersten Datum in 5-Jahres-Abständen alle Datumswerte durchgehend
years = [i for i in range(df.date[1], df.date[end], step=Year(5))]
# Erstelle einen Array mit passenden labels bei dem nur die Jahreszahl angegeben wird 
year_label = ["$year" for year in range(year(df.date[1]), year(df.date[end]), step=5)]

plot(x, y,
    label="Crude oil, Brent",
    xlabel="year",
    ylabel="price in dollar \$",
    xlims=extrema(x),
    xticks=(years, year_label),
    legend=:topleft,
)

#### Aufgabe 2
1. Erstelle einen Plot, der 
- die Zeitreihen der Preise für die Rohölsorten Brent, Dubai, WTI und den Average  beinhaltet,
- ab dem Jahr 1980 bis heute läuft und als alle fünf Jahre ein x-label hat, das nur das Jahr (statt dem vollen Datum) darstellt,
- passende Achsen- und Legendenlabels hat,
- ein passendes Theme verwendet.
2. Erstelle einen zweiten Plot der die Differenz der Rohölsorten darstellt, beschrifte beide Plots sinnvoll und ordne diesen untereinander in einem einzigen Plot an. 
